<a href="https://colab.research.google.com/github/Mafatihul/newsUADscraping/blob/main/newsUADScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount drive
# Kode pogram ini memungkinkan kamu untuk menyambungkan gdrive mu ke notebook ini
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Run program ini sebelum menjalankan blok di bawahnya
# Kode ini menginstall dua library yang dibutuhkan untuk melakukan web sraping
!pip install selenium
!pip install bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [3]:
# Kode program ini mengimport berbagai library yang diperlukan untuk melakukan tugas dibawah
# sehingga library tersebut dapat digunakan
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import pandas as pd

# Fungsi untuk mengambil link dari halaman berita terkini
Untuk mengambil artikel pada news uad, kita perlu memiliki url setiap judul berita. Fungsi ini bisa digunakan untuk mengambil kumpulan link berita dari halaman berita terkini pada news.uad.ac.id. Kamu dapat mengkostumisasi atau mengubah informasi yang ingin anda dapatkan dari web tersebut dengan mengubah beberapa kode program pada fungsi ini. Tetapi ingat untuk menganalisis tag html pada news.uad.ac.id untuk mendapat informasi yang diinginkan. Fungsi link scraper memiliki 2 parameter, yaitu url dan pages. url merupakan link web yang ingin diambil datanya, pages adalah jumlah halaman yang ingin diambil datanya. Tapi fungsi ini tidak bisa digunakan untuk semua website, kamu harus melakukan beberapa perubahan pada fungsi untuk melakukan hal tersebut.

In [18]:
def link_scraper(url, pages):
  try:
    # Variabel untuk menyimpan hasil scraping link
    newsArchives = []
    page = 1

    # Perulangan ini digunakan untuk melakukan pengambilan data pada setiap halaman
    # Banyak halaman yang ingin diambil datanya didefinisikan melalui parameter pages
    while page <= pages:
      # Configure WebDriver to use headless Firefox
      options = Options()
      options.add_argument('-headless')
      driver = webdriver.Firefox(options=options)
      # Membuka link halaman web yang dituju
      link = url + str(page)
      driver.get(link)


      # Selenium will wait for a maximum of 5 seconds for an element matching the given criteria to be found.
      # If no element is found in that time, Selenium will raise an error.
      try:
        wait = WebDriverWait(driver, timeout=5)
        wait.until(EC.presence_of_element_located((By.ID, 'bdpp-post-timeline-1')))
      except:
        raise LookupError("There is no element specified")

      # BeautifulSoup will parse the URL
      content = driver.page_source
      soup = BeautifulSoup(content, 'html.parser')

      # BeautifulSoup will find the CSS class that contain product container
      # Get the text from the specified element and assign them to the variables
      for news in soup.find_all('div', class_='bdpp-tmln-content'):
        news_headline = news.find('h2', class_='bdpp-post-title').text
        news_url = news.find('a', href=True).get('href')
        news_date = news.find('span', class_='bdpp-post-date').text

        # Menambahkan data yang sudah didapat ke dalam variabel newsArchives
        newsArchives.append(
            {
                'Judul': news_headline,
                'Urls': news_url,
                'Terbit': news_date
            }
          )
      page+=1

      # Close the WebDriver
      driver.quit()
    df = pd.DataFrame(newsArchives)
    return df

  except Exception as e:
    print('An error occured: ', e)

    # Close the webdriver
    driver.quit()

# Fungsi untuk mengambil artikel
Fungsi ini digunakan untuk mengambil artikel dalam setiap link yang sudah kita ambil dari fungsi sebelumnya. Fungsi ini menghasilkan sekumpulan file txt berisi judul dan isi berita.

In [5]:
def news_scraper(urls):
  i=0
  for url in urls:
    i+=1
    try:
      # Configure WebDriver to use headless Firefox
      options = Options()
      options.add_argument('-headless')
      driver = webdriver.Firefox(options=options)
      # Get the URL given
      driver.get(url)

      # BeautifulSoup will parse the URL
      content = driver.page_source
      soup = BeautifulSoup(content, 'html.parser')

      # BeautifulSoup will find the CSS class that contain product container
      # Get the text from the specified element and assign them to the variables
      article = soup.find('article')
      if article :
        headline = article.find('h1').text
        paragraph = article.find_all('p')
        # Menyimpan artikel ke file teks
        # Ini dapat kamu ubah sesuai dengan lokasi folder tertentu dimana kamu ingin menyimpan file-file artikelmu
        directory = f'/content/drive/MyDrive/TFD Prep/articles/article{i}.txt'
        # Membuat file txt dan menuliskan h
        with open(directory, 'w', encoding='utf-8') as file:
          file.write(headline)
          file.write('\n')
          for p in paragraph:
            file.write(p.get_text(strip=True))
            file.write('\n')
          # Jika artikel berhasil diambil, program akan mencetak lokasi file txt artikel tersebut disimpan
          print(f'Artikel disimpan pada: {directory}')
      else:
        # url kosong atau artikel tidak ditemukan, program akan mencetak tulisan di bawah
        print(f'Tidak ada paragraf ditemukan di {url}')

      # Menutup web driver
      driver.quit()

    except Exception as e:
      # Jika terjadi error pada fungsi ini, kode ini akan dijalankan
      # Program akan memberitahukan error yang terjadi
      print('An error occured: ', e)

      # Close the webdriver
      driver.quit()

# Menerapkan kedua fungsi
Kode program di bawah digunakan untuk menerapkan kedua file tersebut sehingga kita mendapat data yang dibutuhkan. Hasil dari fungsi pertama merupakan file excel yang memuat link berita. Fungsi kedua menghasilkan file-file yang disimpan dalam folder yang diinginkan sesuai path pada variabel directory.

In [21]:
# Mengambil link dari halaman arsip berita terkini
# Link utama untuk mengambil link berita lain disimpan dalam pandas datafram
main_url = 'https://news.uad.ac.id/terkini/page/'
news_archives = link_scraper(main_url, 3) # Jumlah halaman bisa disesuaikan
news_archives

,Judul,Urls,Terbit
0,"Shaldan, Mahasiswa FK Raih Juara I pada Call f...",https://news.uad.ac.id/shaldan-mahasiswa-fk-ra...,18/10/2024
1,Mahasiswi Teknik Kimia UAD Angkat Isu Pendidik...,https://news.uad.ac.id/mahasiswi-teknik-kimia-...,18/10/2024
2,Program Studi Akuntansi UAD Adakan Seminar Lit...,https://news.uad.ac.id/program-studi-akuntansi...,18/10/2024
3,LPSI UAD Adakan Baitul Arqam Pegawai Baru UAD ...,https://news.uad.ac.id/lpsi-uad-adakan-baitul-...,17/10/2024
4,"Intan Awalia Putri, Raih Prestasi di Ahmad Dah...",https://news.uad.ac.id/intan-awalia-putri-raih...,17/10/2024
5,UKM Badminton UAD Borong Juara di Turnamen se-...,https://news.uad.ac.id/ukm-badminton-uad-boron...,16/10/2024
6,PBS UAD Gelar Kuliah Umum dan Seminar Nasional...,https://news.uad.ac.id/pbs-uad-gelar-kuliah-um...,16/10/2024
7,Dosen UAD Dilantik sebagai Anggota Kolegium Ke...,https://news.uad.ac.id/dosen-uad-diangkat-sump...,15/10/2024
8,UKM PSM Ahda Gitana Gelar Mini Concert Pra-Kom...,https://news.uad.ac.id/ukm-psm-ahda-gitana-gel...,15/10/2024
9,"Himafi UAD Adakan Physics Got Talent 2024, Fas...",https://news.uad.ac.id/himafi-uad-adakan-physi...,15/10/2024


In [22]:
# Save the DataFrame to an Excel file
# File path bisa diganti sesuai kebutuhan
news_archives.to_excel('/content/drive/MyDrive/TFD Prep/news_archives.xlsx', index=False)

In [ ]:
# Menjalankan fungsi kedua untuk mengambil atikel setiap
urls = news_archives['Urls']
news_scraper(urls)

Artikel disimpan pada: /content/drive/MyDrive/TFD Prep/articles/article1.txt
Artikel disimpan pada: /content/drive/MyDrive/TFD Prep/articles/article2.txt
Artikel disimpan pada: /content/drive/MyDrive/TFD Prep/articles/article3.txt
Artikel disimpan pada: /content/drive/MyDrive/TFD Prep/articles/article4.txt
Artikel disimpan pada: /content/drive/MyDrive/TFD Prep/articles/article5.txt
Artikel disimpan pada: /content/drive/MyDrive/TFD Prep/articles/article6.txt
An error occured:  Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//news.uad.ac.id/himafi-uad-adakan-physics-got-talent-2024-fasilitasi-minat-mahasiswa-di-bidang-non-akademik/&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20news.uad.ac.id.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:832:5
checkReadyState@chrome://rem

P.S. Kalau mau costumize source code ini tapi masih bingung, tanya aja langsung